In [1]:
# keras_007-part2_chap4-simple_mnist_example_early_stopped_model_with_patience
#
# 김태영, 블록과 함께 하는 파이썬 딥러닝 케라스
# Part2. 딥러닝 개념잡기
# Chapter4. 학습 조기종료 시키기
# 2018-10-04 (목)

# p.61, 2. 조기종료 시키기 + patence
# 학습 조기종료를 위해서 EarlyStopping()이라는 함수를 사용
# 콜백함수란 어떤 함수를 수행 시 그 함수에서 내가 지정한 함수를 호출하는 것을 말함.
# 이 예제에서는 fit()함수에서 EarlyStopping() 콜백함수가 학습과정 중에 매번 호출됨.
# 지정된 최대 Epoch가 있어도, 학습과정에서 EarlyStopping 콜백함수를 호출하여 해당 조건이 되면 학습을 조기종료 시킵니다.
#
# EarlyStopping 콜백함수에서 patence인자를 사용합니다.
# patience는 개선이 없다고 바로 종료하지 않고, 개선이 없는 에포크를 얼마나 기다려줄 것인가를 설정할 수 있습니다.
# 만약 20이라고 지정하면 개선이 없는 에포크가 20번째 지속될 경우 학습을 종료합니다.

# 0. 사용할 패키지 불러오기
import keras
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

#1. 데이터셋 생성하기
# 훈련셋과 시험셋 불러오기
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 훈련셋과 검증셋 분리
x_val = x_train[50000:]
y_val = y_train[50000:]
x_train = x_train[:50000]
y_train = y_train[:50000]

# 데이터셋 전처리
x_train = x_train.reshape(50000,784).astype('float32') / 255.0
x_val = x_val.reshape(10000,784).astype('float32') / 255.0
x_test = x_test.reshape(10000,784).astype('float32') / 255.0

# 훈련셋과 검증셋 고르기
# random indexs
np.random.seed(3)
train_rand_idxs = np.random.choice(5000,700)
val_rand_idxs = np.random.choice(10000, 300)

x_train = x_train[ train_rand_idxs ]
y_train = y_train[ train_rand_idxs ]
x_val = x_val[ val_rand_idxs ]
y_val = y_val[ val_rand_idxs ]

# 라벨데이터 one-hot encoding 처리
y_train = np_utils.to_categorical( y_train )
y_val = np_utils.to_categorical( y_val )
y_test = np_utils.to_categorical( y_test )

# 2. 모델 구성하기
model = Sequential()
model.add( Dense( units = 2, input_dim = 28*28, activation = 'relu') )
model.add( Dense( units = 10, activation = 'softmax') )

# 3. 모델 학습과정 설정하기
model.compile( loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'] )

# 4. 모델 학습 시키기
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping( patience=20 )

epochs_max = 3000
hist = model.fit( x_train, y_train, epochs = epochs_max, batch_size = 10, validation_data = ( x_val, y_val ), callbacks=[early_stopping])

# 5. 학습과정 살펴보기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot( hist.history['loss'],'y', label='train loss')
loss_ax.plot( hist.history['val_loss'],'r', label='validation loss')
loss_ax.plot( hist.history['acc'],'y', label='train accuracy')
loss_ax.plot( hist.history['val_acc'],'y', label='validation accuracy')

loss_ax.set_xlabel( 'epoch' )
loss_ax.set_ylabel( 'loss' )
acc_ax.set_ylabel( 'accuracy' )

loss_ax.legend( loc = 'upper left' )
acc_ax.legend( loc = 'lower left' )

plt.show()

# 6. 모델 평가하기
loss_and_metrics = model.evaluate( x_test, y_test, batch_size = 32 )

print('')
print('loss : ' + str( loss_and_metrics[0] ) )
print('loss : ' + str( loss_and_metrics[1] ) )

Using TensorFlow backend.


Train on 700 samples, validate on 300 samples


TypeError: 'type' object cannot be interpreted as an integer